# Cross-response functions

$$ r_{i} (t,\tau) = \log m_{i}(t + \tau) - \log m_{i}(t) = \log \frac{m_{i}(t + \tau)}{m_{i}(t)} $$
$$ R_{ij}^{(exc. 0)}(\tau) = \left \langle r_i(t,\tau) \epsilon_{j} (t) \right \rangle_t^{(exc. 0)} $$

In [1]:
# Import modules

import numpy as np
import pickle
import os

from matplotlib import pyplot as plt

from itch_data_generator import trade_sign_reshape

__tau__ = 1000

In [2]:
def cross_response_data(ticker_i, ticker_j, day, tau_val, t_step):
    """
    Obtain the cross response function using the midpoint log returns of
    ticker i and trade signs of ticker j during different time lags. The data
    is adjusted to use only the values each t_step ms
        :param ticker_i: string of the abbreviation of the midpoint stock to
         be analized (i.e. 'AAPL')
        :param ticker_j: string of the abbreviation of the trade sign stock to
         be analized (i.e. 'AAPL')
        :param day: string of the day to be analized (i.e '07')
        :param tau_val: maximum time lag to be analyzed
        :param t_step: time step in the data in ms
    """

    print('Cross response function data')
    print('Processing data for the stock i ' + ticker_i + ' and stock j ' +
          ticker_j + ' the day ' + day + ' March, 2016')
    print('Time step: ', t_step, 'ms')

    # Load data
    midpoint_i = pickle.load(open(
                '../Data/midpoint_data/midpoint_201603{}_{}.pickl'
                .format(day, ticker_i), 'rb'))
    trade_sign_j = pickle.load(open(
                '../Data/trade_signs_data/trade_signs_most_201603{}_{}.pickl'
                .format(day, ticker_j), 'rb'))
    time = pickle.load(open('../Data/midpoint_data/time.pickl', 'rb'))

    # Setting variables to work with t_step ms accuracy

    # Array of the average of each tau. 10^3 s used by Wang
    cross_response_tau = np.zeros(__tau__)

    # Using values each second
    midpoint_i_sec = midpoint_i[::t_step]
    # Changing time from 1 ms to t_step ms
    time_t_step = time[::t_step]

    # reshape and average data of trade signs
    trade_sign_j_sec_avg, trade_sign_j_sec_nr = trade_sign_reshape(
                                                trade_sign_j, time_t_step)

    # Calculating the midpoint log return and the cross response function

    # Depending on the ta
    for tau_idx, tau_v in enumerate(range(1, tau_val + 1, int(tau_val * 1E-3))):

        # Obtain the midpoint log return. Displace the numerator tau values to
        # the right and compute the return, and append the remaining values of
        # tau with zeros
        log_return_i_sec = np.append(np.log(
            midpoint_i_sec[tau_v:]/midpoint_i_sec[:-tau_v]), np.zeros(tau_v))

        cross_response_tau[tau_idx] = np.mean(
            log_return_i_sec[trade_sign_j_sec_nr != 0] *
            trade_sign_j_sec_avg[trade_sign_j_sec_nr != 0])

    # Saving data

    if (not os.path.isdir('../Data/cross_response_data_{}ms/'.format(t_step))):

        os.mkdir('../Data/cross_response_data_{}ms/'.format(t_step))
        print('Folder to save data created')

    pickle.dump(cross_response_tau, open(
        '../Data/cross_response_data_{}ms/cross_201603{}_{}i_{}j_{}ms.pickl'
        .format(t_step, day, ticker_i, ticker_j, t_step), 'wb'))

    print('Cross response function data saved')
    print()

    return None

In [3]:
# Using the only two common stocks in both data sets
ticker_i = 'AAPL'
ticker_j = 'MSFT'
days = ['07', '08', '09', '10', '11']
tau_val = 1000
t_step = 1000

In [4]:
for day in days:
    cross_response_data(ticker_i, ticker_j, day, tau_val, t_step)

Cross response function data
Processing data for the stock i AAPL and stock j MSFT the day 07 March, 2016
Time step:  1000 ms
Cross response function data saved

Cross response function data
Processing data for the stock i AAPL and stock j MSFT the day 08 March, 2016
Time step:  1000 ms
Cross response function data saved

Cross response function data
Processing data for the stock i AAPL and stock j MSFT the day 09 March, 2016
Time step:  1000 ms
Cross response function data saved

Cross response function data
Processing data for the stock i AAPL and stock j MSFT the day 10 March, 2016
Time step:  1000 ms
Cross response function data saved

Cross response function data
Processing data for the stock i AAPL and stock j MSFT the day 11 March, 2016
Time step:  1000 ms
Cross response function data saved

